In [ ]:
import os
import logging
import pandas as pd
from pandas.tseries.offsets import BMonthEnd, BYearBegin
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

basedir = os.path.abspath(os.path.dirname(__file__))
webdir = os.path.join(basedir, os.pardir, 'www')

sns.set(style='whitegrid', palette='colorblind', rc={'legend.frameon': True})
plt_config = {'table': True, 'grid': True, 'marker': 'o', 'figsize': (12.53, 6.8)}

def import_from_ksm(engine, tbl):
    df = pd.read_sql_query('SELECT * FROM %s' % tbl, engine, parse_dates=['date'])
    return df

def dates_to_dict(df, field='date'):
    dates = {}
    sorted_df = df.sort_values(by=field)
    result = sorted_df.drop_duplicates(subset=field, keep='last')[field]
    result = pd.DatetimeIndex(result)
    dates['last'] = result.max()
    dates['eopm'] = result.asof(dates['last'] - BMonthEnd())
    dates['boy'] = result.asof(dates['last'] - BYearBegin())
    dates['yago'] = result.asof(dates['last'].replace(year=dates['last'].year-1))
    logger.debug('last => %s', dates['last'])
    logger.debug('eopm => %s', dates['eopm'])
    logger.debug('boy => %s', dates['boy'])
    logger.debug('yago => %s', dates['yago'])
    return dates

def main():
    engine = create_engine('sqlite:///' + os.path.join(basedir, os.pardir, 'ksm.db'))

    for table in ['taxable', 'taxexempt']:
        df = import_from_ksm(engine, table)
        dates = dates_to_dict(df)
        
        # plot most recent credit curves
        most_rec_df = df[df['date'] == dates['last']]
        credit_curv = most_rec_df.pivot('scale', 'rating', 'value')
        credit_curv = credit_curv[['AAA', 'AA', 'A', 'BAA']]
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.set_title('%s MMD Curves as of %s' % (table.title(), dates['last'].strftime('%A, %B %d, %Y')))
        ax.get_xaxis().set_visible(False)
        credit_curv.plot(ax=ax, **plt_config)
        for col in credit_curv.columns:
            for i, val in enumerate(credit_curv[col]):
                if col == 'AAA':
                    ax.annotate(str(val), (i + 1, val), textcoords="offset points", xytext=(0, -10), ha='center', fontsize=6)
                elif col == 'AA':
                    ax.annotate(str(val), (i + 1, val), textcoords="offset points", xytext=(0, 6), ha='center', fontsize=6)
                else:
                    ax.annotate(str(val), (i + 1, val), textcoords="offset points", xytext=(0, 5), ha='center', fontsize=6)
        ax.legend(loc='upper left', shadow=True, fancybox=True)
        plt.savefig(os.path.join(webdir, 'img', table + '_mr.png'), bbox_inches='tight', dpi=150)

        # plot historical, individual curves
        date_series = [dates['last'], dates['eopm'], dates['boy'], dates['yago']]
        ratings_df = df[df['date'].isin(date_series)]
        
        for rating in ['BAA', 'A', 'AA', 'AAA']:
            rating_df = ratings_df[ratings_df['rating'] == rating]
            rating_df = rating_df.pivot('scale', 'date', 'value')
            rating_df.rename(columns={dates['last']: 'Last', dates['eopm']: 'Prior EOM', dates['boy']: 'Beg Yr', dates['yago']: 'Yr Ago'}, inplace=True)
            fig = plt.figure()
            ax = fig.add_subplot(1, 1, 1)
            ax.set_title('%s %s Curve as of %s' % (table.title(), rating, dates['last'].strftime('%A, %B %d, %Y')))
            ax.get_xaxis().set_visible(False)
            rating_df.plot(ax=ax, **plt_config)
            if rating == 'BAA':
                ymax = ax.get_ylim()[1]
            ax.set_ylim((0, ymax))
            
            # Add labels above the line
            for col in rating_df.columns:
                for i, val in enumerate(rating_df[col]):
                    if col == 'Prior EOM':
                        xytext = (0, -15)  # Move the "Prior EOM" labels lower
                    elif col == 'Last':
                        xytext = (0, 6)   # Move the "Last" labels slightly higher
                    else:
                        xytext = (0, 5)   # Default position for other categories
                    ax.annotate(
                        str(val), (i + 1, val), textcoords="offset points", 
                        xytext=xytext, ha='center', fontsize=6)
            ax.legend(loc='upper left', shadow=True, fancybox=True)
            plt.savefig(os.path.join(webdir, 'img', table +'_' + rating + '.png'), bbox_inches='tight', dpi=150)
        
        for rating in ['BAA', 'A', 'AA']:
            dt_rng = pd.bdate_range(end=dates['last'], periods=125, freq='D')
            # Select date range
            df = df[df['date'].isin(dt_rng)]
            # Select Base df
            base_df = df[df['rating'] == 'AAA']
            # transform
            base_df = base_df.pivot(index='date', columns='scale', values='value')
            # Select Minuend df
            minuend_df = df[df['rating'] == rating]
            # transform
            minuend_df = minuend_df.pivot(index='date', columns='scale', values='value')
            # Calculate spreads
            spread_df = (minuend_df - base_df) * 100.0
            most_recent_df = spread_df.loc[dates['last'],:]
            most_recent_df.name = 'Last'

            fig = plt.figure(figsize=(12.53, 6.8))
            ax = fig.add_subplot(1, 1, 1)
            ax.set_title("%s %s-%s Spreads from %s to %s" % (table.title(), rating, 'AAA', dt_rng[0].strftime('%m-%d-%Y'), dates['last'].strftime('%m-%d-%Y')))
            ax.get_xaxis().set_visible(False)
            sns.violinplot(data=spread_df, color='#DAE8F5', positions=[1], alpha=0.5, ax=ax)
            
            # Now, add data labels for most_recent_df with rounding to 2 decimal places above markers ('v')
            for i, val in enumerate(most_recent_df):
                xytext = (0, 10)  # Adjust this value as needed
                rounded_val = round(val, 2)  # Round to 2 decimal places
                if i == len(most_recent_df) // 2:
                    # If it's the middle value, adjust the label position
                    xytext = (0, 20)  # Move the label higher for the middle value
                ax.annotate(
                    str(rounded_val), (i, val), textcoords="offset points", 
                    xytext=xytext, ha='center', fontsize=6)
            plt.savefig(os.path.join(webdir, 'img', table +'_spreads_' + rating + '.png'), bbox_inches='tight', dpi=150)

if __name__ == '__main__':
    import sys
    sys.exit(main())

